In [1]:
import pandas as pd
import json
import re

def clean_output(output):
    cleaned_output = output
    # Extract XML code from the output
    xml_code = re.search(r'```json(.*?)```', output, re.DOTALL)
    cleaned_output = ''
    if xml_code:
        cleaned_output = xml_code.group(1).strip()
    
    # Parse the JSON
    try:
        # cleaned_output = json.loads(cleaned_output)
        cleaned_output = eval(cleaned_output)
    except:
        return {}
    
    return cleaned_output

prompts = pd.read_json('fn1.7-test-prompts.jsonl', lines=True)

# predictions = pd.read_json('batch-gpt-4o-mini-0.0temperature-json-existing-cot-predictions-cleaned.jsonl', lines=True)
with open('batch-ft:gpt-4o-mini-2024-07-18:idir-lab:fn1-7-json-existing-smalltrain:AVo7xNhv-0.0temperature-json-existing-predictions-cleaned.jsonl') as f:
    predictions = f.readlines()

predictions = [json.loads(p.strip()) for p in predictions]

prompts['prediction'] = predictions
prompts['output'] = prompts.messages.apply(lambda x: clean_output(x[-1]['content']))
predictions = prompts[['output', 'prediction']]

In [3]:
tp = 0
near_miss = 0
fp = 0
fn = 0
tn = 0 # not really used because tags only have positive values, technically all other FEs are TN, but we don't really care about that (maybe we should?)

for label, pred, fes in predictions[['output', 'prediction', 'output']].values:

    # Get each predicted FE span from the prediction
    pred_tags = pred
    real_tags = fes

    # Check each predicted FE span
    for tag, content in pred_tags.items():
        if tag.capitalize() in real_tags:
            if content == real_tags[tag.capitalize()]:
                tp += 1
            else:
                # print(f'Near miss: {content} vs {real_tags[tag.capitalize()]}')
                fp += 1
                near_miss += 1
        else:
            fp += 1
    
    # Check each real FE span
    for tag, content in real_tags.items():
        if tag not in pred_tags:
            fn += 1
        elif content != pred_tags[tag]:
            fn += 1

# print(f'Perfect: {perfect} out of {len(predictions)}')
print(f'TP: {tp}')
print(f'Missed: {near_miss}')
print(f'FP: {fp}')
print(f'FN: {fn}')

print(f'Precision (ignoring near misses): {tp / (tp + fp - near_miss)}')
print(f'Precision: {tp / (tp + fp)}')
print(f'Recall: {tp / (tp + fn)}')
print(f'F1: {2 * tp / (2 * tp + fp + fn)}')

print(f'Accuracy: {tp / (tp + fn + fp)}')



TP: 7392
Missed: 1336
FP: 2913
FN: 3557
Precision (ignoring near misses): 0.8241721485115397
Precision: 0.7173216885007277
Recall: 0.6751301488720431
F1: 0.6955867130893009
Accuracy: 0.5332563843601212
